In [45]:
import requests
import json
import math


def get_content_betano(game_url):
    r = requests.get(game_url)
    content = r.text
    return content


def get_bet_event(game_url):
    content = get_content_betano(game_url)
    token_mark = '<script>window["initial_state"]='
    inx_start = content.find(token_mark) + len(token_mark)
    inx_end = content.find("</script>", inx_start)
    json_content = content[inx_start:inx_end]
    dic_data = json.loads(json_content)
    bet_event = dic_data["data"]["event"]
    return bet_event


def get_players(event):
    match = event["name"]
    separator = " - "
    player1, player2 = str.split(match, separator)
    return player1, player2


def find_by_name(value, lst):
    return list(filter(lambda x: x["name"] == value, lst))


def get_bet_target(token_class, bet_player, markets):
    winner_markets = find_by_name(token_class, markets)
    if len(winner_markets) > 0:
        winner_market = winner_markets[0]
    else:
        raise Exception(f"ERROR: {token_class} not found")

    tracked_player_list = find_by_name(bet_player, winner_market["selections"])
    if len(tracked_player_list) > 0:
        tracked_player_obj = tracked_player_list[0]
    else:
        raise Exception(f"ERROR: {bet_player} not found")
    return tracked_player_obj


def find_by_name(value, lst):
    return list(filter(lambda x: x["name"] == value, lst))


def get_bet_target(token_class, bet_player, markets):
    winner_markets = find_by_name(token_class, markets)
    if len(winner_markets) > 0:
        winner_market = winner_markets[0]
    else:
        raise Exception(f"ERROR: {token_class} not found")

    tracked_player_list = find_by_name(bet_player, winner_market["selections"])
    if len(tracked_player_list) > 0:
        tracked_player_obj = tracked_player_list[0]
    else:
        raise Exception(f"ERROR: {bet_player} not found")
    return tracked_player_obj


def has_triggered_bet(tracked_player_obj, bet_value_expected):
    if tracked_player_obj["price"] >= bet_value_expected:
        msg = f'$ {tracked_player_obj["name"]} alcancou {bet_value_expected}: Atual {tracked_player_obj["price"]}'
        return msg


def is_second_half(event):
    try:
        period = event["liveData"]["periodDescription"]
        if period != "P1" and period != "P2":
            return True
    except:
        return


def can_bet_for_total_points_on_2nd_half(event):
    if is_second_half(event):
        first = event["liveData"]["results"]["firstPeriodScore"]
        second = event["liveData"]["results"]["secondPeriodScore"]
        firstHome, firstAway = first["home"], first["away"]
        secondHome, secondAway = second["home"], second["away"]
        totalFirstHalf = (
            int(firstHome) + int(firstAway) + int(secondHome) + int(secondAway)
        )
        markets = event["markets"]
        total_points = find_by_name("Total de Pontos", markets)
        less_than_points_obj = total_points[0]["selections"][1]
        handicap = less_than_points_obj["handicap"]
        projectedPoints = totalFirstHalf * 2
        can_bet = "NO" if projectedPoints - 3 >= handicap else "YES"
        # print(f"IT'S SECOND HALF - TOTAL POINTS:  -> can bet? {can_bet}")
        if can_bet == "YES":
            return "$ JOGAR EM TOTAL DE PONTOS PELO 2o Tempo: {totalFirstHalf}*2={projectedPoints} - hand: {handicap}"


from nba_api.stats.endpoints import *
import pandas


class TeamStats:
    def __init__(self, team_name, pontos):
        self.name = team_name
        self.media = pontos.mean()
        self.media_ultima = pontos[:10].mean()
        self.hot = pontos.sort_values(ascending=False)[:10].mean()
        self.cold = pontos.sort_values()[:10].mean()

    def pnts_esperados(self):
        res = (self.hot + self.media_ultima) / 2
        return math.floor(res)

    def pnts_esperados_baixo(self):
        res = (self.cold + self.media_ultima) / 2
        return math.floor(res)

    def __str__(self):
        return f"{self.name}:{self.pnts_esperados_baixo()}<-->{self.pnts_esperados()}"


def get_team_stats(team):
    teams_data = leaguedashteamstats.LeagueDashTeamStats().get_data_frames()[0]
    team_id_res = teams_data[teams_data["TEAM_NAME"] == team]["TEAM_ID"]
    team_id = team_id_res.values[0]
    pontos = teamgamelogs.TeamGameLogs(
        season_nullable="2023-24", team_id_nullable=team_id
    ).get_data_frames()[0]["PTS"]
    return TeamStats(team, pontos)


def bet_pnts_by_player(player_stats, markets):
    try:
        player_tot_pnts_markets = find_by_name(f"Total de Pontos {player_stats.name}", markets)
        player_tot_pnts = player_tot_pnts_markets[0]["selections"]
        
        player_tot_pnts = player_tot_pnts + player_tot_pnts_markets[1]["selections"]
        player_tot_pnts = list(sorted(player_tot_pnts, key=lambda x: x["handicap"]))
    except:
        pass
    
    
    menor_pnts_do_jogo_provavel_player = player_stats.pnts_esperados_baixo()
    pnts_do_jogo_provavel_player1 = player_stats.pnts_esperados()
    player_tot_pnts_mais_de = [o for o in player_tot_pnts if "Mais de" in o["name"]]
    player_tot_pnts_menos_de = [o for o in player_tot_pnts if "Menos de" in o["name"]]
    
    for i in range(len(player_tot_pnts_mais_de)):
        maior_que_curr = player_tot_pnts_mais_de[i]
        aposta_maior_que_curr = maior_que_curr["handicap"]
        if aposta_maior_que_curr+1 > menor_pnts_do_jogo_provavel_player:
            preco = maior_que_curr["price"]
            print(
                f"$ JOGAR TOT_PNTS(ALT){player_stats.name} MAIOR QUE: {aposta_maior_que_curr}->{preco}/ESPERADO MAIOR: {menor_pnts_do_jogo_provavel_player}"
            )
            break
    for i in range(len(player_tot_pnts_menos_de)):
        menor_que_curr = player_tot_pnts_menos_de[i]
        aposta_menor_que_curr = menor_que_curr["handicap"]
        if aposta_menor_que_curr > pnts_do_jogo_provavel_player1:
            preco = menor_que_curr["price"]
            print(
                f"$ JOGAR TOT_PNTS(ALT){player_stats.name} MENOR QUE: {aposta_menor_que_curr }->{preco}/ESPERADO MENOR: {pnts_do_jogo_provavel_player1}"
            )
            break
    return player_tot_pnts


def print_json(dic):
    print(json.dumps(dic))

import pandas as pd
from datetime import datetime as dt

ZEBRA_GAME_FILENAME = "zebras_game.csv"

def to_date_str(x):
    return str(x.date()).replace("-", "")

def get_betano_games():
    betano_games_url = "https://br.betano.com/sport/basquete/eua/nba/441g/"
    content = get_content_betano(betano_games_url)
    marca_inicio = '<script type="application/ld+json">'
    indice_inicio = content.find(marca_inicio) + len(marca_inicio)
    indice_fim = content.find("</script>", indice_inicio)
    json_content = content[indice_inicio:indice_fim]
    dic_data = json.loads(json_content)

    return dic_data


def find_zebras_game():
    betano_games = get_betano_games()
    zebra_games = []
    for game in betano_games:
        url = game["url"]
        evento = get_bet_event(url)
        markets = evento["markets"]

        aposta_vencedor = find_by_name("Vencedor", markets)[0]

        apostas = aposta_vencedor["selections"]
        player1 = apostas[0]
        player2 = apostas[1]
        if player1["price"] > 3 or player2["price"] > 3:
            # print(f"! ZEBRA: {player1['name']} X {player2['name']} {url}")
            o = {}
            o["startTime"] = evento["startTime"]
            o["name"] = evento["name"]
            o["id"] = int(evento["id"])
            zebra = player1 if player1["price"] > 3 else player2
            o["zebra_name"] = zebra["name"]
            o["zebra_price"] = zebra["price"]
            zebra_games.append(o)
    return zebra_games

def save_betano_zebras():
    zebra_games = find_zebras_game()
    
    startDateCol = "startDate"
    try:
        df_csv = pd.read_csv(ZEBRA_GAME_FILENAME, index_col="startDate")
    except:
        df_csv = pd.DataFrame(
            columns=[
                startDateCol,
                "startTime",
                "name",
                "id",
                "zebra_name",
                "zebra_price",
            ]
        )
        df_csv = df_csv.set_index("startDate")
    df = pandas.DataFrame()
    df2 = df.from_records(zebra_games)
    df_to_date = lambda x: dt.fromtimestamp(x / 1000)
    df2["startDate"] = [to_date_str(df_to_date(i)) for i in df2["startTime"].values]
    df2 = df2.set_index("startDate")

    ids_existent = df_csv.id.values
    df_csv_news = df2[~df2["id"].isin(ids_existent)]

    df_csv_save = pd.concat([df_csv, df_csv_news])
    df_csv_save.to_csv(ZEBRA_GAME_FILENAME)

def get_zebra_from_today():
    df_csv = pd.read_csv(ZEBRA_GAME_FILENAME)
    today = dt.today()
    today_inx= int(to_date_str(today))
    df_zebra_today =  df_csv[df_csv['startDate']==today_inx]
    return df_zebra_today

# get_zebra_from_today()
# save_betano_zebras()


In [15]:
def get_today_betano_games_urls():
    betano_games = get_betano_games()
    todays_game_url = []
    today = dt.today().date()
    for game in betano_games:
        time_obj_game = dt.fromisoformat(game['startDate'])
        date_game  = time_obj_game.astimezone().date()
        if date_game == today:
            todays_game_url.append(game['url']+'?bt=8')
        # break
    return todays_game_url

['https://br.betano.com/odds/toronto-raptors-chicago-bulls/44106080/?bt=8',
 'https://br.betano.com/odds/new-york-knicks-washington-wizards/44106062/?bt=8',
 'https://br.betano.com/odds/utah-jazz-oklahoma-city-thunder/44106168/?bt=8']

In [4]:
import time
from plyer import notification
import pygame
import ctypes  # An included library with Python install.
import winsound

# Inicializa o mixer do pygame para sons
pygame.mixer.init()

def enviar_alerta(msg):
    # Exibe um popup
    # notification.notify(
    #     title='Alerta',
    #     message=msg,
    #     app_name='Temporizador'
    # )
    
    Mbox.BoxOkCancelAsync("ALERTA BET", msg)

    # Toca um som de alerta
    pygame.mixer.music.load('alarm.mp3')
    pygame.mixer.music.play()


class Mbox:
    OK = 1
    CANCELADO = 2

    def BoxOkCancel(title, text):
        winsound.MessageBeep(winsound.MB_OK)
        ctypes.windll.user32.MessageBoxW(0, text, title, 1)

    def Alerta(title, text=""):
        winsound.MessageBeep(winsound.MB_OK)
        funcao = lambda: ctypes.windll.user32.MessageBoxW(0, text, title, 0x1000)
        Mbox.executar_alerta_paralelo(funcao)

    def executar_alerta_paralelo(funcao_alerta, *args):
        thread_exec = ThreadSimples(funcao_alerta, *args)
        thread_exec.start()

    def BoxOkCancelAsync(title, text):
        Mbox.executar_alerta_paralelo(Mbox.BoxOkCancel, title, text)


pygame 2.5.2 (SDL 2.28.3, Python 3.11.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [5]:
import threading
import time
TEMPO_ALARME   =  30 # SEGUNDOS
class ThreadCustom(threading.Thread):
    def __init__(self, tempo, executar_funcao):
        super(ThreadCustom, self).__init__()
        self.parar = threading.Event()
        self.tempo = tempo
        self.executar = executar_funcao

    def run(self):
        while not self.parar.is_set():
            self.executar()
            time.sleep(self.tempo)

    def parar_execucao(self):
        print("Parando a execução...")
        self.parar.set()

def parar_thread(thread):
    thread.parar_execucao()
    thread.join()  # Aguarde até que a thread termine completamente

class ThreadSimples(threading.Thread):
    def __init__(self, funcao, *args, **kwargs):
        # If you are going to have your own constructor, make sure you call the parent constructor too!
        #        Thread.__init__(self)
        super().__init__()
        self.funcao = funcao
        self.kwargs = kwargs
        self.args = args

    def run(self):
        self.funcao(*self.args, **self.kwargs)

['https://br.betano.com/odds/toronto-raptors-chicago-bulls/44106080/?bt=8',
 'https://br.betano.com/odds/new-york-knicks-washington-wizards/44106062/?bt=8',
 'https://br.betano.com/odds/utah-jazz-oklahoma-city-thunder/44106168/?bt=8']

In [46]:
try:
    if len(betano_url_games) ==0:
        betano_url_games = get_today_betano_games_urls()
except:
    betano_url_games = get_today_betano_games_urls()


def main():
    for url in betano_url_games:
        print(">>>>")
        process_game(url)

def process_game(game_url):
    try:
        event = get_bet_event(game_url)
    except:
        print(f"ERROR ON: {game_url}")
        return
    player1_name, player2_name = get_players(event)
    p1_stats = get_team_stats(player1_name)
    p2_stats = get_team_stats(player2_name)
    pnts_do_jogo_provavel = p1_stats.pnts_esperados() + p2_stats.pnts_esperados()
    menor_pnts_do_jogo_provavel = (
        p1_stats.pnts_esperados_baixo() + p2_stats.pnts_esperados_baixo()
    )
    tot_pnts_str = f"TOT_PNTS: {math.floor(menor_pnts_do_jogo_provavel)}<-->{math.floor(pnts_do_jogo_provavel)}"
    print(f"> {player1_name} X {player2_name}: {tot_pnts_str}: {game_url}")
    bet_value_expected = 3
    markets = event["markets"]
    token_winner = "Vencedor"
    zebras = get_zebra_from_today()
    zebras_name = zebras.zebra_name.values
    p2_is_zebra  = player2_name in zebras_name
    p1_is_zebra  = player1_name in zebras_name
    if p1_is_zebra or p2_is_zebra:
        track_favorite = player2_name if p1_is_zebra else player1_name
        tracked_player_favorite_obj = get_bet_target(token_winner, track_favorite, markets)
        
    # print(f'{token_winner}:{tracked_player_obj["name"]}:{tracked_player_obj["price"]}')
        result = has_triggered_bet(tracked_player_favorite_obj, bet_value_expected)
        if result:
            enviar_alerta(result)

    bet_on_2nd_half = can_bet_for_total_points_on_2nd_half(event)
    if bet_on_2nd_half:
        enviar_alerta(bet_on_2nd_half)

    total_de_pontos = find_by_name("Total de Pontos", markets)
    menos_de_total_de_pontos = total_de_pontos[0]["selections"][1]
    aposta_tot_pnts = menos_de_total_de_pontos["handicap"]

    if aposta_tot_pnts > pnts_do_jogo_provavel:
        preco = menos_de_total_de_pontos["price"]
        print(f"$ JOGAR TOT_PNTS APOSTA: {aposta_tot_pnts}->{preco}")
    try:
        total_de_pontos_alternativas = total_de_pontos[1]
    except:
        total_de_pontos_alternativas = find_by_name(
            "Total de Pontos (alternativas)", markets
        )[0]
    rows = total_de_pontos_alternativas["tableLayout"]["rows"]

    preco_maior_que = lambda x: x["groupSelections"][0]["selections"][0]
    for i in range(len(rows)):
        maior_que_curr = preco_maior_que(rows[i])
        aposta_maior_que_curr = maior_que_curr["handicap"]
        if aposta_maior_que_curr + 1 > menor_pnts_do_jogo_provavel:
            preco = maior_que_curr["price"]
            print(f"$ JOGAR TOT_PNTS(ALT) MAIOR QUE: {aposta_maior_que_curr}->{preco}/ESPERADO MAIOR: {menor_pnts_do_jogo_provavel}")
            break

    preco_menor_que = lambda x: x["groupSelections"][0]["selections"][1]
    for i in range(len(rows)):
        menor_que_curr = preco_menor_que(rows[i])
        aposta_menor_que_curr = menor_que_curr["handicap"]
        if aposta_menor_que_curr > pnts_do_jogo_provavel:
            preco = menor_que_curr["price"]
            print(f"$ JOGAR TOT_PNTS(ALT) MENOR QUE: {aposta_menor_que_curr }->{preco}/ESPERADO MENOR: {pnts_do_jogo_provavel}")
            break
    print(p1_stats)    
    print(p2_stats)    
    bet_pnts_by_player( p1_stats, markets)
    bet_pnts_by_player( p2_stats, markets)
    

main()

>>>>
> Toronto Raptors X Chicago Bulls: TOT_PNTS: 213<-->243: https://br.betano.com/odds/toronto-raptors-chicago-bulls/44106080/?bt=8
$ JOGAR TOT_PNTS(ALT) MAIOR QUE: 212.5->1.23/ESPERADO MAIOR: 213
$ JOGAR TOT_PNTS(ALT) MENOR QUE: 243.5->1.24/ESPERADO MENOR: 243
Toronto Raptors:110<-->126
Chicago Bulls:103<-->117
$ JOGAR TOT_PNTS(ALT)Toronto Raptors MAIOR QUE: 109.5->1.45/ESPERADO MAIOR: 110
$ JOGAR TOT_PNTS(ALT)Chicago Bulls MAIOR QUE: 102.5->1.12/ESPERADO MAIOR: 103
$ JOGAR TOT_PNTS(ALT)Chicago Bulls MENOR QUE: 117.5->1.55/ESPERADO MENOR: 117
>>>>
> New York Knicks X Washington Wizards: TOT_PNTS: 211<-->241: https://br.betano.com/odds/new-york-knicks-washington-wizards/44106062/?bt=8
$ JOGAR TOT_PNTS(ALT) MAIOR QUE: 214.5->1.17/ESPERADO MAIOR: 211
$ JOGAR TOT_PNTS(ALT) MENOR QUE: 241.5->1.47/ESPERADO MENOR: 241
New York Knicks:106<-->121
Washington Wizards:105<-->120
$ JOGAR TOT_PNTS(ALT)New York Knicks MAIOR QUE: 110.5->1.12/ESPERADO MAIOR: 106
$ JOGAR TOT_PNTS(ALT)New York Knicks 

In [ ]:

# thread = ThreadCustom(TEMPO_ALARME,main)
# thread.start()